In [9]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [10]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, IntegerType
from pyspark.sql.functions import col,expr
from pyspark.sql.functions import sum,count
from pyspark.sql.functions import stddev_pop
from pyspark.sql.functions import max
from pyspark.sql.functions import mean ,corr,max, min,stddev_pop ,stddev_samp,covar_pop,covar_samp
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import RFormula
spark = SparkSession.builder.getOrCreate()
sc =SparkContext.getOrCreate()


In [11]:
zoo_data = spark.read.option("inferschema","true").option("header","true").csv("zoo.csv")
zoo_data.show(5)

+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+
|AnimalName|Hair|Feathers|Eggs|Milk|Airborne|Aquatic|Predator|Toothed|Backbone|Breathes|Venomous|Fins|Legs|Tail|Domestic|Catsize|Type|
+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+
|  aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|
|  antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|   1|
|      bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|   4|
|      bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|
|      boar|   1|       0|   0|   1|       0|      0|  

In [12]:
zoo_data = zoo_data.withColumn("ISMammal",expr("case when type == 1 then 1 else 0 end") )
zoo_data.show(5)

+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+--------+
|AnimalName|Hair|Feathers|Eggs|Milk|Airborne|Aquatic|Predator|Toothed|Backbone|Breathes|Venomous|Fins|Legs|Tail|Domestic|Catsize|Type|ISMammal|
+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+--------+
|  aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|       1|
|  antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|   1|       1|
|      bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|   4|       0|
|      bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|  

In [27]:
preprocessed_data = RFormula(formula="ISMammal ~ Hair + Feathers + Eggs + Milk + Airborne + Aquatic + Predator +  Toothed + Backbone + Breathes + Venomous + Fins + Legs + Tail + Domestic + Catsize")
preprocessed_data = preprocessed_data.fit(zoo_data)
preprocessed_data = preprocessed_data.transform(zoo_data)


In [34]:
preprocessed_data.select("features","label").show(10) 

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(16,[0,3,6,7,8,9,...|  1.0|
|(16,[0,3,7,8,9,12...|  1.0|
|(16,[2,5,6,7,8,11...|  0.0|
|(16,[0,3,6,7,8,9,...|  1.0|
|(16,[0,3,6,7,8,9,...|  1.0|
|(16,[0,3,7,8,9,12...|  1.0|
|(16,[0,3,7,8,9,12...|  1.0|
|(16,[2,5,7,8,11,1...|  0.0|
|(16,[2,5,6,7,8,11...|  0.0|
|(16,[0,3,7,8,9,12...|  1.0|
+--------------------+-----+
only showing top 10 rows



In [29]:
train , test =preprocessed_data.randomSplit([0.7,0.3])


In [30]:
#configure classifier
lr = LogisticRegression(labelCol = "label",featuresCol = "features")

In [31]:
#train classifier
fittedLR = lr.fit(train)

In [35]:
result = fittedLR.transform(test)
result.select("AnimalName","label","prediction").show()


+----------+-----+----------+
|AnimalName|label|prediction|
+----------+-----+----------+
|      boar|  1.0|       1.0|
|   chicken|  0.0|       0.0|
|      chub|  0.0|       0.0|
|   dogfish|  0.0|       0.0|
|      duck|  0.0|       0.0|
|  elephant|  1.0|       1.0|
|  flamingo|  0.0|       0.0|
|      flea|  0.0|       0.0|
|      frog|  0.0|       0.0|
|      frog|  0.0|       0.0|
|   gorilla|  1.0|       1.0|
|   haddock|  0.0|       0.0|
|  honeybee|  0.0|       0.0|
|      kiwi|  0.0|       0.0|
|      lion|  1.0|       1.0|
|      lynx|  1.0|       1.0|
|      moth|  0.0|       0.0|
|   octopus|  0.0|       0.0|
|   opossum|  1.0|       1.0|
|      oryx|  1.0|       1.0|
+----------+-----+----------+
only showing top 20 rows

